In [1]:
import pandas as pd
import numpy as np

In [2]:
players = pd.read_csv("player_list.csv")["ID"]

## Data Cleaning (adjust dates to numeric)

In [8]:


months = {"OCT": "10", "NOV": "11", "DEC": "12", "JAN": "01", "FEB": "02", "MAR": "03", "APR": "04"}


for i,  player in enumerate(players):
    if i % 100 == 0:
        print(i)
    file = pd.read_csv(str(player) + ".csv")
    for i in range(file.shape[0]):
        month = months[file["GAME_DATE"][i][:3]]
        day = file["GAME_DATE"][i][4:6]
        year = file["GAME_DATE"][i][-4:]
        date = str(year) + str(month) + str(day)
        file["GAME_DATE"][i] = int(date) 
    file = file.drop(columns = ["Unnamed: 0"])
    file.to_csv(str(player) + ".csv")

0


/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


100
200
300
400
500
600
700
800
900
1000


## Data Cleaning (segement data by season)

In [11]:
for j, player in enumerate(players):
    if j % 100 == 0:
        print(j)
    season = pd.Series([])
    file = pd.read_csv(str(player) + ".csv")
    for i in range(file.shape[0]):
        if file["GAME_DATE"][i] < 20140500:
            season[i] = int(2014)
        elif file["GAME_DATE"][i] < 20150500:
            season[i] = int(2015)
        elif file["GAME_DATE"][i] < 20160500:
            season[i] = int(2016)
        elif file["GAME_DATE"][i] < 20170500:
            season[i] = int(2017)
        elif file["GAME_DATE"][i] < 20180500:
            season[i] = int(2018)
        elif file["GAME_DATE"][i] < 20190500:
            season[i] = int(2019)
        elif file["GAME_DATE"][i] < 20200500:
            season[i] = int(2020)

    file.insert(2, "Season", season)
    file = file.drop(columns = ["Unnamed: 0"])
    file.to_csv(str(player) + ".csv")

0


/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  after removing the cwd from sys.path.


100
200
300
400
500
600
700
800
900
1000


## Compile Datasets

**Feature computation for a given team over all games in dataset over given time period for short and long-term**

In [390]:
def computeDataBulkTeam(team, short, long, term):
    
    #Check
    checker = pd.Series([], dtype="int64")
    
    #Chemistry
    average_games = pd.Series([], dtype="float64")
    p_2pt_assisted = pd.Series([], dtype="float64")
    p_3pt_assisted = pd.Series([], dtype="float64")
    ast_tov= pd.Series([], dtype="float64")
    team_tov = pd.Series([], dtype="float64")
    passes = pd.Series([], dtype="float64")
    
    #Team Averages
    ppg = pd.Series([], dtype="float64")
    pitp = pd.Series([], dtype="float64")
    ppg_2pt = pd.Series([], dtype="float64")
    ppg_3pt = pd.Series([], dtype="float64")
    ppg_2nd_chance = pd.Series([], dtype="float64")
    ppg_ft = pd.Series([], dtype="float64")
    ppg_fb = pd.Series([], dtype="float64")
    p_ft = pd.Series([], dtype="float64")
    efg = pd.Series([], dtype="float64")
    ts = pd.Series([], dtype="float64")
    pace = pd.Series([], dtype="float64")
    off_rat = pd.Series([], dtype="float64")
    p_win = pd.Series([], dtype="float64")
    reb_p = pd.Series([], dtype="float64")
    fta_rate = pd.Series([], dtype="float64")
    assists = pd.Series([], dtype="float64")
    blocks = pd.Series([], dtype="float64")
    rebounds = pd.Series([], dtype="float64")
    pfd = pd.Series([], dtype="float64")
    
    #Shot distribution
    p_ofPts_2pt = pd.Series([], dtype="float64")
    p_ofPts_3pt = pd.Series([], dtype="float64")
    p_ofPts_fb = pd.Series([], dtype="float64")
    p_ofPts_tov = pd.Series([], dtype="float64")
    p_ofPts_paint = pd.Series([], dtype="float64")
    p_ofPts_ft = pd.Series([], dtype="float64")
    p_ofPts_mr = pd.Series([], dtype="float64")
    p_ofA_2pt = pd.Series([], dtype="float64")
    p_ofA_3pt = pd.Series([], dtype="float64")

    #Opponent
    opp_steals = pd.Series([], dtype="float64")
    opp_blocks = pd.Series([], dtype="float64")
    opp_reb = pd.Series([], dtype="float64")
    opp_def_rat = pd.Series([], dtype="float64")
    opp_p_win = pd.Series([], dtype="float64")
    opp_p_TS = pd.Series([], dtype="float64")
    opp_pf = pd.Series([], dtype="float64")
    opp_tov = pd.Series([], dtype="float64")
    opp_reb_p = pd.Series([], dtype="float64")
    opp_cfga = pd.Series([], dtype="float64")
    opp_cfg_p = pd.Series([], dtype="float64")
    
    #Opponent Indices
    _3ptM_ind = pd.Series([], dtype="float64")
    _paint_ind = pd.Series([], dtype="float64")
    _ft_ind = pd.Series([], dtype="float64")

    file = pd.read_csv(str(team) + ".csv")
    games = pd.read_csv("Master_GameList.csv")
    players = pd.read_csv("player_list.csv")
    for i in range(file.shape[0]):
        if i >= short:
            if term == "long":
                lower = max(0, i - long)
            else:
                lower = i - short
                
            checker[i - short] = i
            
            #Game identifier
            game_id = file["GAME_ID"].iloc[i]
            game = games[games["GAME_ID"] == game_id]
            game = game[game["TEAM_ABBREVIATION"] == team]
            
            #Opponent information
            if game["Away"].iloc[0] == team:
                opp_team_name = game["Home"].iloc[0]
            else:
                opp_team_name = game["Away"].iloc[0]
            opp_team = pd.read_csv(opp_team_name + ".csv")
            opp_idx = opp_team.index[opp_team["GAME_ID"] == game_id].tolist()[0]
            if term == "long":
                opp_bound = max(0, i - long)
            else:
                opp_bound = max(0, i - short)
                
            if term == "long":
                team_file = pd.read_csv(str(team) + str(long) + "_special.csv")
            else:
                team_file = pd.read_csv(str(team) + str(short) + "_special.csv")
                
    
            
            #Chemistry
            num_games = 0
            divisor = 0
            game_file = pd.read_csv(str(game_id) + ".csv")
            game_file = game_file[game_file["TEAM_ABBREVIATION"] == team][:5]
            for player in game_file["PLAYER_ID"].tolist():
                if player in players["ID"].tolist():
                    player_file = pd.read_csv(str(int(player)) + ".csv")
                    num_games = player_file[player_file["TEAM_ABBREVIATION"] == team].shape[0]
                    divisor += 1
            average_games[i - short] = num_games / divisor
            
            p_2pt_assisted[i - short] = np.mean(file["PCT_AST_2PM"][lower:i])
            p_3pt_assisted[i - short] = np.mean(file["PCT_AST_3PM"][lower:i])
            ast_tov[i - short] = np.mean(file["AST_TOV"][lower:i])
            team_tov[i - short] = np.mean(file["TM_TOV_PCT"][lower:i])
            passes[i - short] = np.mean(file["PASS"][lower:i])
        
            
            #Team Averages
            ppg[i - short] = np.mean(file["PTS"][lower:i])
            pitp[i - short] = np.mean(file["PTS_PAINT"][lower:i])
            ppg_2pt[i - short] = np.mean(file["PCT_PTS_2PT"][lower:i] * file["PTS"][lower:i])
            ppg_3pt[i - short] = np.mean(file["PCT_PTS_3PT"][lower:i] * file["PTS"][lower:i])
            ppg_2nd_chance[i - short] = np.mean(file["PTS_2ND_CHANCE"][lower:i])
            ppg_ft[i - short] = np.mean(file["FTM"][lower:i])
            ppg_fb[i - short] = np.mean(file["PTS_FB"][lower:i])
            p_ft[i - short] = np.mean(file["FT_PCT"][lower:i])
            efg[i - short] = np.mean(file["EFG_PCT"][lower:i])
            ts[i - short] = np.mean(file["TS_PCT"][lower:i])
            pace[i - short] = np.mean(file["E_PACE"][lower:i])
            off_rat[i - short] = np.mean(file["OFF_RATING"][lower:i])
            p_win[i - short] = team_file["p_win"].iloc[i - short]
            reb_p[i - short] = np.mean(file["REB_PCT"][lower:i])
            fta_rate[i - short] = np.mean(file["FTA_RATE"][lower:i])
            assists[i - short] = np.mean(file["AST"][lower:i])
            blocks[i - short] = np.mean(file["BLK"][lower:i])
            rebounds[i - short] = np.mean(file["REB"][lower:i])
            pfd[i - short] = np.mean(file["PFD"][lower:i])
            
            #Shot Distribution
            p_ofPts_2pt[i - short] = np.mean(file["PCT_PTS_2PT"][lower:i])
            p_ofPts_3pt[i - short] = np.mean(file["PCT_PTS_3PT"][lower:i])
            p_ofPts_fb[i - short] = np.mean(file["PCT_PTS_FB"][lower:i])
            p_ofPts_tov[i - short] = np.mean(file["PCT_PTS_OFF_TOV"][lower:i])
            p_ofPts_paint[i - short] = np.mean(file["PCT_PTS_PAINT"][lower:i])
            p_ofPts_ft[i - short] = np.mean(file["PCT_PTS_FT"][lower:i])
            p_ofPts_mr[i - short] = np.mean(file["PCT_PTS_2PT_MR"][lower:i])
            p_ofA_2pt[i - short] = np.mean(file["PCT_FGA_2PT"][lower:i])
            p_ofA_3pt[i - short] = np.mean(file["PCT_FGA_3PT"][lower:i])
            
            #Opponent information
            opp_steals[i - short] = np.mean(opp_team["STL"][opp_bound:opp_idx])
            opp_blocks[i - short] = np.mean(opp_team["BLK"][opp_bound:opp_idx])
            opp_reb[i - short] = np.mean(opp_team["REB"][opp_bound:opp_idx])
            opp_def_rat[i - short] = np.mean(opp_team["DEF_RATING"][opp_bound:opp_idx])
            opp_p_win[i - short] = team_file["opp_p_win"].iloc[i - short]
            opp_p_TS[i - short] = np.mean(opp_team["OPP_TS_PCT"][opp_bound:opp_idx])
            opp_pf[i - short] = np.mean(opp_team["PF"][opp_bound:opp_idx])
            opp_tov[i - short] = np.mean(opp_team["OPP_TO"][opp_bound:opp_idx])
            opp_reb_p[i - short] = np.mean(opp_team["REB_PCT"][opp_bound:opp_idx])
            opp_cfga[i - short] = np.mean(opp_team["CFGA"][opp_bound:opp_idx])
            opp_cfg_p[i - short] = np.mean(opp_team["CFG_PCT"][opp_bound:opp_idx])
            
            #Opponent indices
            _3ptM_ind[i - short] = np.mean(file["FG3M"][lower:i]) / np.mean(file["OPP_FG3M"][opp_bound:opp_idx])
            _paint_ind[i - short] = np.mean(file["PTS_PAINT"][lower:i]) / np.mean(file["OPP_PTS_PAINT"][opp_bound:opp_idx])
            _ft_ind[i - short] = np.mean(file["FTM"][lower:i]) / np.mean(file["OPP_FTM"][opp_bound:opp_idx])
    
    
    team_data = pd.concat([checker, average_games, p_2pt_assisted, p_3pt_assisted, ast_tov, team_tov, 
                           passes, ppg, pitp, ppg_2pt, ppg_3pt, ppg_2nd_chance, ppg_ft, ppg_fb, p_ft, 
                           efg, ts, pace, off_rat, p_win,  reb_p, fta_rate, assists, blocks, 
                           rebounds, pfd, p_ofPts_2pt, p_ofPts_3pt, p_ofPts_fb, p_ofPts_tov, p_ofPts_paint, 
                           p_ofPts_ft, p_ofPts_mr, p_ofA_2pt, p_ofA_3pt, opp_steals, opp_blocks, opp_reb, 
                           opp_def_rat, opp_p_win, opp_p_TS, opp_pf, opp_tov, opp_reb_p, opp_cfga, opp_cfg_p,
                           _3ptM_ind, _paint_ind, _ft_ind ], axis = 1)
    
    if term == "short":
        team_data.columns = ["s_checker", "s_average_games", "s_p_2pt_assisted", "s_p_3pt_assisted", 
                             "s_ast_tov", "s_team_tov", "s_passes", "s_ppg", "s_pitp", "s_ppg_2pt", 
                             "s_ppg_3pt", "s_ppg_2nd_chance", "s_ppg_ft", "s_ppg_fb", "s_p_ft", 
                             "s_efg", "s_ts", "s_pace", "s_off_rat", "s_p_win", "s_reb_p", 
                             "s_fta_rate", "s_assists", "s_blocks", "s_rebounds", "s_pfd", "s_p_ofPts_2pt",
                             "s_p_ofPts_3pt", "s_p_ofPts_fb", "s_p_ofPts_tov", "s_p_ofPts_paint", 
                             "s_p_ofPts_ft", "s_p_ofPts_mr", "s_p_ofA_2pt", "s_p_ofA_3pt", "s_opp_steals", 
                             "s_opp_blocks", "s_opp_reb", "s_opp_def_rat", "s_opp_p_win", "s_opp_p_TS", 
                             "s_opp_pf", "s_opp_tov", "s_opp_reb_p", "s_opp_cfga", "s_opp_cfg_p", "s_3ptM_ind", 
                             "s_paint_ind", "s_ft_ind"]
        
    else:
        team_data.columns = ["l_checker", "l_average_games", "l_p_2pt_assisted", "l_p_3pt_assisted", 
                             "l_ast_tov", "l_team_tov", "l_passes", "l_ppg", "l_pitp", "l_ppg_2pt", 
                             "l_ppg_3pt", "l_ppg_2nd_chance", "l_ppg_ft", "l_ppg_fb", "l_p_ft", 
                             "l_efg", "l_ts", "l_pace", "l_off_rat", "l_p_win", "l_reb_p", 
                             "l_fta_rate", "l_assists", "l_blocks", "l_rebounds", "l_pfd", "l_p_ofPts_2pt", 
                             "l_p_ofPts_3pt", "l_p_ofPts_fb", "l_p_ofPts_tov", "l_p_ofPts_paint", 
                             "l_p_ofPts_ft", "l_p_ofPts_mr", "l_p_ofA_2pt", "l_p_ofA_3pt", "l_opp_steals", 
                             "l_opp_blocks", "l_opp_reb", "l_opp_def_rat", "l_opp_p_win", "l_opp_p_TS", 
                             "l_opp_pf", "l_opp_tov", "l_opp_reb_p", "l_opp_cfga", "l_opp_cfg_p", "l_3ptM_ind", 
                             "l_paint_ind", "l_ft_ind"]
        
    team_data = team_data.replace([np.inf, -np.inf, np.nan], 0)
    return team_data

**Game Identifiers as well as none time period dependent features**

In [398]:
def oneTimeTeam(team, short, long):
    
    #Game Indicators
    Game = pd.Series([], dtype="int64")
    Date = pd.Series([], dtype="int64")
    Home = pd.Series([], dtype="object")
    Away = pd.Series([], dtype="object")
    Team = pd.Series([], dtype="object")
    
    #Y
    y = pd.Series([], dtype="int64")
    
    #Rest
    b2b = pd.Series([], dtype="float64")
    min_short = pd.Series([], dtype="float")
    home_away = pd.Series([], dtype = "float64")
    
    #Availability
    PPG_avail = pd.Series([], dtype="float64")
    MIN_avail = pd.Series([], dtype="float64")
    
    games = pd.read_csv("Master_GameList.csv")
    file = pd.read_csv(str(team) + ".csv")
    for i in range(file.shape[0]):
        if i >= short:
            lower = i - short 
            
            #Game Indicators
            Game[i - short] = file["GAME_ID"].iloc[i] 
            Date[i - short] = file["Date"].iloc[i] 
            Team[i - short] = file["TEAM_ABBREVIATION"].iloc[i]
            Home[i - short] = file["Home"].iloc[i]
            Away[i - short] = file["Away"].iloc[i]
            
            #Y
            y[i - short] = file["PTS"][i]
            
            #Rest
            min_s = 0
            games_short = file[lower:i]
            for s in range(games_short.shape[0]):
                game_file = pd.read_csv(str(games_short["GAME_ID"].iloc[s]) + ".csv")
                game_file = game_file[game_file["TEAM_ABBREVIATION"] == team][:5]
                for row in range(game_file.shape[0]):
                    min_s += float(game_file["MIN"].iloc[row].replace(":", "."))
            min_short[i - short] = (min_s / short) / 5
            
            if file["Home"].iloc[i] == team:
                home_away[i - short] = 1
            else:
                home_away[i - short] = 0

            availability = pd.read_csv(team + "_avail.csv")[short - 2:]
            
            PPG_avail[i - short] = availability["PPG_avail_" + str(short)].iloc[i - short]
            MIN_avail[i - short] = availability["MIN_avail_" + str(short)].iloc[i - short]
     
    one_time = pd.concat([Game, Date, Home, Away, Team, y, b2b, min_short, home_away,
                          PPG_avail, MIN_avail], axis = 1)
    
    one_time.columns = ["Game", "Date", "Home", "Away", "Team"," y", "b2b", "min_short",
                        "home_away", "PPG_avail", "MIN_avail"]
    
    one_time = one_time.replace([np.inf, -np.inf, np.nan], 0)
    return one_time

**Methods to compute datasets for all teams**

In [399]:
teams = ["PHI", "BOS", "NYK", "BKN", "UTA",
         "DEN", "DAL", "POR", "CLE", "CHI", 
         "LAL", "LAC", "WAS", "CHA", "NOP",
         "GSW", "SAC", "SAS", "HOU", "MEM", 
         "MIN", "IND", "ORL", "OKC", "MIL", 
         "MIA", "DET", "PHX", "ATL", "TOR"]

def computeSetTeam(short, long, term):
    frame = computeDataBulkTeam(teams[0], short, long, term)
    for i, team in enumerate(teams[1:]):
        print("{} % complete".format(round(100* (i / len(teams)), 2)))
        new_frame = computeDataBulkTeam(team, short, long, term)
        frame = pd.concat([frame, new_frame], axis = 0)
    return frame

def computeOneTimeSetTeam(short):
    frame = oneTimeTeam(teams[0], short, long)
    for i, team in enumerate(teams[1:]):
        print("{} % complete".format(round(100* (i / len(teams)), 2)))
        new_frame = oneTimeTeam(team, short, long)
        frame = pd.concat([frame, new_frame], axis = 0)
    return frame

**Compute short-term features**

In [401]:
st = []
for i in range(2, 11):
    st.append(computeSetTeam(i, 50, "short"))
    print("Done with short for i = {}".format(i))

0.0 % complete
3.33 % complete
6.67 % complete
10.0 % complete
13.33 % complete
16.67 % complete
20.0 % complete
23.33 % complete
26.67 % complete


/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:165: RuntimeWarning: divide by zero encountered in double_scalars


30.0 % complete
33.33 % complete
36.67 % complete
40.0 % complete
43.33 % complete
46.67 % complete
50.0 % complete
53.33 % complete
56.67 % complete
60.0 % complete
63.33 % complete
66.67 % complete
70.0 % complete
73.33 % complete
76.67 % complete
80.0 % complete
83.33 % complete
86.67 % complete
90.0 % complete
93.33 % complete
Done with short for i = 2
0.0 % complete
3.33 % complete
6.67 % complete
10.0 % complete
13.33 % complete
16.67 % complete
20.0 % complete
23.33 % complete
26.67 % complete
30.0 % complete
33.33 % complete
36.67 % complete
40.0 % complete
43.33 % complete
46.67 % complete
50.0 % complete
53.33 % complete
56.67 % complete
60.0 % complete
63.33 % complete
66.67 % complete
70.0 % complete
73.33 % complete
76.67 % complete
80.0 % complete
83.33 % complete
86.67 % complete
90.0 % complete
93.33 % complete
Done with short for i = 3
0.0 % complete
3.33 % complete
6.67 % complete
10.0 % complete
13.33 % complete
16.67 % complete
20.0 % complete
23.33 % complete
26.67

**Compute long-term stats**

In [402]:
lt = []
for i in range(40, 81, 5):
    lt.append(computeSetTeam(2, i, "long"))
    print("Done with long for i = {}".format(i))

0.0 % complete
3.33 % complete
6.67 % complete
10.0 % complete
13.33 % complete
16.67 % complete
20.0 % complete
23.33 % complete
26.67 % complete
30.0 % complete
33.33 % complete
36.67 % complete
40.0 % complete
43.33 % complete
46.67 % complete
50.0 % complete
53.33 % complete
56.67 % complete
60.0 % complete
63.33 % complete
66.67 % complete
70.0 % complete
73.33 % complete
76.67 % complete
80.0 % complete
83.33 % complete
86.67 % complete
90.0 % complete
93.33 % complete
Done with long for i = 40
0.0 % complete
3.33 % complete
6.67 % complete
10.0 % complete
13.33 % complete
16.67 % complete
20.0 % complete
23.33 % complete
26.67 % complete
30.0 % complete
33.33 % complete
36.67 % complete
40.0 % complete
43.33 % complete
46.67 % complete
50.0 % complete
53.33 % complete
56.67 % complete
60.0 % complete
63.33 % complete
66.67 % complete
70.0 % complete
73.33 % complete
76.67 % complete
80.0 % complete
83.33 % complete
86.67 % complete
90.0 % complete
93.33 % complete
Done with long

**Compute game identifiers/none time period dependent stats**

In [403]:
ot = []
for i in range(2, 11):
    ot.append(computeOneTimeSetTeam(i))
    print("Done with one time for i = {}")

0.0 % complete
3.33 % complete
6.67 % complete
10.0 % complete
13.33 % complete
16.67 % complete
20.0 % complete
23.33 % complete
26.67 % complete
30.0 % complete
33.33 % complete
36.67 % complete
40.0 % complete
43.33 % complete
46.67 % complete
50.0 % complete
53.33 % complete
56.67 % complete
60.0 % complete
63.33 % complete
66.67 % complete
70.0 % complete
73.33 % complete
76.67 % complete
80.0 % complete
83.33 % complete
86.67 % complete
90.0 % complete
93.33 % complete
Done with one time for i = {}
0.0 % complete
3.33 % complete
6.67 % complete
10.0 % complete
13.33 % complete
16.67 % complete
20.0 % complete
23.33 % complete
26.67 % complete
30.0 % complete
33.33 % complete
36.67 % complete
40.0 % complete
43.33 % complete
46.67 % complete
50.0 % complete
53.33 % complete
56.67 % complete
60.0 % complete
63.33 % complete
66.67 % complete
70.0 % complete
73.33 % complete
76.67 % complete
80.0 % complete
83.33 % complete
86.67 % complete
90.0 % complete
93.33 % complete
Done with 

**Compute defensive stats for a given team in the short and long term**

In [439]:
def computeDefensive(team, short, long, term):
    checker = pd.Series([], dtype = "int64")
    steals = pd.Series([], dtype = "float64")
    def_rat = pd.Series([], dtype = "float64")
    pf = pd.Series([], dtype = "float64")
    opp_tov = pd.Series([], dtype = "float64")
    cfga = pd.Series([], dtype = "float64")
    cfg_p = pd.Series([], dtype = "float64")
    ppg_against = pd.Series([], dtype = "float64")
    ppg_against_3pt = pd.Series([], dtype = "float64")
    ppg_against_paint = pd.Series([], dtype = "float64")
    ppg_against_ft = pd.Series([], dtype = "float64")
    p_efg_against = pd.Series([], dtype = "float64")
    
    file = pd.read_csv(str(team) + ".csv")
    
    for i in range(file.shape[0]):
        if i >= short:
            if term == "long":
                lower = max(0, i - long)
            else:
                lower = i - short
                
            checker[i - short] = i  
            
            steals[i - short] = np.mean(file["STL"][lower:i])
            def_rat[i - short] = np.mean(file["DEF_RATING"][lower:i])
            pf[i - short] = np.mean(file["PF"][lower:i])
            opp_tov[i - short] = np.mean(file["OPP_TO"][lower:i])
            cfga[i - short] = np.mean(file["CFGA"][lower:i])
            cfg_p[i - short] = np.mean(file["CFG_PCT"][lower:i])
            ppg_against[i - short] = np.mean(file["OPP_PTS"][lower:i])
            ppg_against_3pt[i - short] = np.mean(file["OPP_FG3M"][lower:i])
            ppg_against_paint[i - short] = np.mean(file["OPP_PTS_PAINT"][lower:i])
            ppg_against_ft[i - short] = np.mean(file["OPP_FTM"][lower:i])
            p_efg_against[i - short] = np.mean(file["OPP_EFG_PCT"][lower:i])
    
    
    def_data = pd.concat([checker, steals, def_rat, pf, opp_tov, cfga, cfg_p, ppg_against, ppg_against_3pt, 
                          ppg_against_paint, ppg_against_ft, p_efg_against], axis = 1)
    
    if term == "short":
        def_data.columns = ["s_checker", "s_steals", "s_def_rat", "s_pf", "s_opp_tov", "s_cfga", "s_cfg_p", 
                            "s_ppg_against", "s_against_3ptm", "s_ppg_against_paint", "s_ppg_against_ft", 
                            "s_p_efg_against"]
        
    else:
        def_data.columns = ["l_checker", "l_steals", "l_def_rat", "l_pf", "l_opp_tov", "l_cfga", "l_cfg_p", 
                            "l_ppg_against", "l_against_3ptm", "l_ppg_against_paint", "l_ppg_against_ft", 
                            "l_p_efg_against"]
        
    def_data = def_data.replace([np.inf, -np.inf, np.nan], 0)
    return def_data
    
    


**Method to compute defensive stats for all teams**

In [443]:
teams = ["PHI", "BOS", "NYK", "BKN", "UTA",
         "DEN", "DAL", "POR", "CLE", "CHI", 
         "LAL", "LAC", "WAS", "CHA", "NOP",
         "GSW", "SAC", "SAS", "HOU", "MEM", 
         "MIN", "IND", "ORL", "OKC", "MIL", 
         "MIA", "DET", "PHX", "ATL", "TOR"]

def computeDefensiveSet(short, long, term):
    frame = computeDefensive(teams[0], short, long, term)
    for i, team in enumerate(teams[1:]):
        new_frame = computeDefensive(team, short, long, term)
        frame = pd.concat([frame, new_frame], axis = 0)
    return frame



**Compile defensive stats for the short term and the long term**

In [444]:
dts = []
for i in range(2, 11):
    dts.append(computeDefensiveSet(i, 50, "short"))
    print("Done with short defensive for i = {}".format(i))
    
dtl = []
for i in range(40, 81, 5):
    dtl.append(computeDefensiveSet(2, i, "long"))
    print("Done with short defensive for i = {}".format(i))


Done with short defensive for i = 2
Done with short defensive for i = 3
Done with short defensive for i = 4
Done with short defensive for i = 5
Done with short defensive for i = 6
Done with short defensive for i = 7
Done with short defensive for i = 8
Done with short defensive for i = 9
Done with short defensive for i = 10
Done with short defensive for i = 40
Done with short defensive for i = 45
Done with short defensive for i = 50
Done with short defensive for i = 55
Done with short defensive for i = 60
Done with short defensive for i = 65
Done with short defensive for i = 70
Done with short defensive for i = 75
Done with short defensive for i = 80


**Concatenate datasets that include only short term stats**

In [457]:
#Compute only short term data sets

short = [2, 3, 4, 5, 6, 7, 8, 9, 10]
compiled_sonly = []

for i in range(len(short)):
    onetime = ot[i].reset_index(drop = True)
    offensive = st[i].reset_index(drop = True).iloc[:, :35]
    defensive = dts[i].reset_index(drop = True)
    print("{} {} {}\n".format(onetime.shape, offensive.shape, defensive.shape))
    frame = pd.concat([onetime, offensive, defensive], axis = 1).drop(columns = ["s_checker"])
    frame = frame.sort_values(by = ["Date", "Game"])
    frame = frame.reset_index(drop = True)
    to_remove = []
    for i in range(frame.shape[0]):
        game_id = frame["Game"].iloc[i]
        game = frame[frame["Game"] == game_id]
        if game.shape[0] < 2:
            to_remove.append(i)
    frame = frame.drop(to_remove)
    compiled_sonly.append(frame)
    

    

(16642, 11) (16642, 35) (16642, 12)

(16612, 11) (16612, 35) (16612, 12)

(16582, 11) (16582, 35) (16582, 12)

(16552, 11) (16552, 35) (16552, 12)

(16522, 11) (16522, 35) (16522, 12)

(16492, 11) (16492, 35) (16492, 12)

(16462, 11) (16462, 35) (16462, 12)

(16432, 11) (16432, 35) (16432, 12)

(16402, 11) (16402, 35) (16402, 12)



**Organize dataset so both teams for a given game are on the same line (currently each team is on its own line)**

In [474]:
merged_sets_sonly = []

for dataset in compiled_sonly:
    print(dataset.shape)
    home_frame = pd.DataFrame(columns = dataset.columns).add_prefix("home_")
    away_frame = pd.DataFrame(columns = dataset.columns).add_prefix("away_")
    for i in range(0, dataset.shape[0], 2):
        if dataset["Team"].iloc[i] == dataset["Home"].iloc[i]:
            append_home = dataset[i: i + 1].add_prefix("home_")
            home_frame = pd.concat([home_frame, append_home], axis = 0)
            
            append_away = dataset[i + 1: i + 2].add_prefix("away_")
            away_frame = pd.concat([away_frame, append_away], axis = 0)
        else:
            append_home = dataset[i + 1: i + 2].add_prefix("home_")
            home_frame = pd.concat([home_frame, append_home], axis = 0)
            
            append_away = dataset[i: i + 1].add_prefix("away_")
            away_frame = pd.concat([away_frame, append_away], axis = 0)
    
    home_frame = home_frame.reset_index(drop = True)
    away_frame = away_frame.reset_index(drop = True)
    frame = pd.concat([home_frame, away_frame], axis = 1)
    print(frame.shape)
    merged_sets_sonly.append(frame)

print(st[0].reset_index(drop = True).iloc[:, :35].head(3))

(16638, 56)
(8319, 112)
(16608, 56)
(8304, 112)
(16574, 56)
(8287, 112)
(16548, 56)
(8274, 112)
(16512, 56)
(8256, 112)
(16480, 56)
(8240, 112)
(16448, 56)
(8224, 112)
(16416, 56)
(8208, 112)
(16390, 56)
(8195, 112)
   s_checker  s_average_games  s_p_2pt_assisted  s_p_3pt_assisted  s_ast_tov  \
0          2             22.2             0.505            0.7710      1.350   
1          3             22.2             0.504            0.7425      1.485   
2          4             22.2             0.511            0.9090      1.195   

   s_team_tov  s_passes  s_ppg  s_pitp  s_ppg_2pt  ...  s_pfd  s_p_ofPts_2pt  \
0       17.95     337.0  111.5    62.0    77.0175  ...   20.5         0.6925   
1       16.75     306.5  108.0    59.0    72.9960  ...   15.0         0.6750   
2       18.60     289.0   98.5    44.0    56.9865  ...   19.0         0.5785   

   s_p_ofPts_3pt  s_p_ofPts_fb  s_p_ofPts_tov  s_p_ofPts_paint  s_p_ofPts_ft  \
0         0.1880        0.2055         0.2055            0.559

**Write dataset to csv**

In [510]:
short = [2, 3, 4, 5, 6, 7, 8, 9, 10]

for i, dataset in enumerate(merged_sets_sonly):
    data_sonly = dataset.drop(columns = ["away_Game", "away_Date", "away_Away", "away_Home", 
                                         "away_Team", "away_ y", "home_ y", "home_Team"])
    data_sonly = data_sonly.rename(columns={"home_Game": "Game", "home_Date": "Date", "home_Home": "Home", 
                                            "home_Away": "Away"})
    
    for j in range(data_sonly.shape[0]):
        game = games[games["Game"] == data_sonly["Game"].iloc[j]]
        game_home = game[game["Team"] == data_sonly["Home"].iloc[j]]
        game_away = game[game["Team"] == data_sonly["Away"].iloc[j]]
        if game_home.shape[0] > 0:
            data_sonly["home_b2b"].iloc[j] = game_home["b2b"].iloc[0]
        if game_away.shape[0] > 0:
            data_sonly["away_b2b"].iloc[j] = game_away["b2b"].iloc[0]
    
    data_sonly.to_csv("TeamDataset_" + str(short[i]) + ".csv")

yep
yep
yep
yep
yep
yep
yep
yep
yep


**Concatenate datasets that include both long and short term data**

In [515]:
#Compute long term data sets

short = [2, 3, 4, 5, 6, 7, 8, 9, 10]
long = [40, 45, 50, 55, 60, 65, 70, 75, 80]
compiled = []

for i in range(len(short)):
    for j in range(len(long)):
        onetime = ot[i].reset_index(drop = True)
        offensive_s = st[i].reset_index(drop = True).iloc[:, :35]
        defensive_s = dts[i].reset_index(drop = True)
        
        offensive_l = lt[j][lt[j]["l_checker"] >= short[i]]
        offensive_l = offensive_l.reset_index(drop = True).iloc[:, :35]
    
        defensive_l = dtl[j][dtl[j]["l_checker"] >= short[i]]
        defensive_l = defensive_l.reset_index(drop = True)
        
        print("{} {} {} {} {}".format(onetime.shape, offensive_s.shape, defensive_s.shape, offensive_l.shape, defensive_l.shape))
        frame = pd.concat([onetime, offensive_s, offensive_l, defensive_s, defensive_l], axis = 1).drop(columns = ["l_checker", "s_checker"])
        frame = frame.sort_values(by = ["Date", "Game"])
        frame = frame.reset_index(drop = True)
        to_remove = []
        for k in range(frame.shape[0]):
            game_id = frame["Game"].iloc[k]
            game = frame[frame["Game"] == game_id]
            if game.shape[0] < 2:
                to_remove.append(k)
        frame = frame.drop(to_remove)
        compiled.append(frame)

(16642, 11) (16642, 35) (16642, 12) (16642, 35) (16642, 12)
(16642, 11) (16642, 35) (16642, 12) (16642, 35) (16642, 12)
(16642, 11) (16642, 35) (16642, 12) (16642, 35) (16642, 12)
(16642, 11) (16642, 35) (16642, 12) (16642, 35) (16642, 12)
(16642, 11) (16642, 35) (16642, 12) (16642, 35) (16642, 12)
(16642, 11) (16642, 35) (16642, 12) (16642, 35) (16642, 12)
(16642, 11) (16642, 35) (16642, 12) (16642, 35) (16642, 12)
(16642, 11) (16642, 35) (16642, 12) (16642, 35) (16642, 12)
(16642, 11) (16642, 35) (16642, 12) (16642, 35) (16642, 12)
yep
(16612, 11) (16612, 35) (16612, 12) (16612, 35) (16612, 12)
(16612, 11) (16612, 35) (16612, 12) (16612, 35) (16612, 12)
(16612, 11) (16612, 35) (16612, 12) (16612, 35) (16612, 12)
(16612, 11) (16612, 35) (16612, 12) (16612, 35) (16612, 12)
(16612, 11) (16612, 35) (16612, 12) (16612, 35) (16612, 12)
(16612, 11) (16612, 35) (16612, 12) (16612, 35) (16612, 12)
(16612, 11) (16612, 35) (16612, 12) (16612, 35) (16612, 12)
(16612, 11) (16612, 35) (16612, 12) 

**Move game stats to one line (as opposed to two lines as it is currently)**

In [516]:
merged_sets = []

for dataset in compiled:
    print(dataset.shape)
    home_frame = pd.DataFrame(columns = dataset.columns).add_prefix("home_")
    away_frame = pd.DataFrame(columns = dataset.columns).add_prefix("away_")
    for i in range(0, dataset.shape[0], 2):
        if dataset["Team"].iloc[i] == dataset["Home"].iloc[i]:
            append_home = dataset[i: i + 1].add_prefix("home_")
            home_frame = pd.concat([home_frame, append_home], axis = 0)
            
            append_away = dataset[i + 1: i + 2].add_prefix("away_")
            away_frame = pd.concat([away_frame, append_away], axis = 0)
        else:
            append_home = dataset[i + 1: i + 2].add_prefix("home_")
            home_frame = pd.concat([home_frame, append_home], axis = 0)
            
            append_away = dataset[i: i + 1].add_prefix("away_")
            away_frame = pd.concat([away_frame, append_away], axis = 0)
    
    home_frame = home_frame.reset_index(drop = True)
    away_frame = away_frame.reset_index(drop = True)
    frame = pd.concat([home_frame, away_frame], axis = 1)
    print(frame.shape)
    merged_sets.append(frame)

(16638, 101)
(8319, 202)
(16638, 101)
(8319, 202)
(16638, 101)
(8319, 202)
(16638, 101)
(8319, 202)
(16638, 101)
(8319, 202)
(16638, 101)
(8319, 202)
(16638, 101)
(8319, 202)
(16638, 101)
(8319, 202)
(16638, 101)
(8319, 202)
(16608, 101)
(8304, 202)
(16608, 101)
(8304, 202)
(16608, 101)
(8304, 202)
(16608, 101)
(8304, 202)
(16608, 101)
(8304, 202)
(16608, 101)
(8304, 202)
(16608, 101)
(8304, 202)
(16608, 101)
(8304, 202)
(16608, 101)
(8304, 202)
(16574, 101)
(8287, 202)
(16574, 101)
(8287, 202)
(16574, 101)
(8287, 202)
(16574, 101)
(8287, 202)
(16574, 101)
(8287, 202)
(16574, 101)
(8287, 202)
(16574, 101)
(8287, 202)
(16574, 101)
(8287, 202)
(16574, 101)
(8287, 202)
(16548, 101)
(8274, 202)
(16548, 101)
(8274, 202)
(16548, 101)
(8274, 202)
(16548, 101)
(8274, 202)
(16548, 101)
(8274, 202)
(16548, 101)
(8274, 202)
(16548, 101)
(8274, 202)
(16548, 101)
(8274, 202)
(16548, 101)
(8274, 202)
(16512, 101)
(8256, 202)
(16512, 101)
(8256, 202)
(16512, 101)
(8256, 202)
(16512, 101)
(8256, 202)


**Write datasets to csv**

In [518]:
names = ["2_40", "2_45", "2_50", "2_55", "2_60", "2_65", "2_70", "2_75", "2_80", 
         "3_40", "3_45", "3_50", "3_55", "3_60", "3_65", "3_70", "3_75", "3_80",
         "4_40", "4_45", "4_50", "4_55", "4_60", "4_65", "4_70", "4_75", "4_80",
         "5_40", "5_45", "5_50", "5_55", "5_60", "5_65", "5_70", "5_75", "5_80",
         "6_40", "6_45", "6_50", "6_55", "6_60", "6_65", "6_70", "6_75", "6_80",
         "7_40", "7_45", "7_50", "7_55", "7_60", "7_65", "7_70", "7_75", "7_80",
         "8_40", "8_45", "8_50", "8_55", "8_60", "8_65", "8_70", "8_75", "8_80",
         "9_40", "9_45", "9_50", "9_55", "9_60", "9_65", "9_70", "9_75", "9_80",
         "10_40", "10_45", "10_50", "10_55", "10_60", "10_65", "10_70", "10_75", "10_80"]

for i, dataset in enumerate(merged_sets):
    print("yep")
    data_full = dataset.drop(columns = ["away_Game", "away_Date", "away_Away", "away_Home", 
                                         "away_Team", "away_ y", "home_ y", "home_Team"])
    data_full = data_full.rename(columns={"home_Game": "Game", "home_Date": "Date", "home_Home": "Home", 
                                            "home_Away": "Away"})
    
    for j in range(data_full.shape[0]):
        game = games[games["Game"] == data_full["Game"].iloc[j]]
        game_home = game[game["Team"] == data_full["Home"].iloc[j]]
        game_away = game[game["Team"] == data_full["Away"].iloc[j]]
        if game_home.shape[0] > 0:
            data_full["home_b2b"].iloc[j] = game_home["b2b"].iloc[0]
        if game_away.shape[0] > 0:
            data_full["away_b2b"].iloc[j] = game_away["b2b"].iloc[0]
    
    data_full.to_csv("TeamDataset_" + names[i] + ".csv")

yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
